## Based on https://github.com/chkwon/TrafficAssignment.jl

In [1]:
include("../Julia_files/load_network_uni_class.jl")

load_ta_network (generic function with 2 methods)

In [2]:
using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

create_graph (generic function with 1 method)

In [3]:
function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

get_vector (generic function with 1 method)

In [4]:
ta_data = load_ta_network("Sioux Falls")

TA_Data("Sioux Falls",24,24,1,76,[1,1,2,2,3,3,3,4,4,4  …  22,22,22,22,23,23,23,24,24,24],[2,3,1,6,1,4,12,3,5,11  …  15,20,21,23,14,22,24,13,21,23],[25900.2,23403.5,25900.2,4958.18,23403.5,17110.5,23403.5,17110.5,17782.8,4908.83  …  9599.18,5075.7,5229.91,5000.0,4924.79,5000.0,5078.51,5091.26,4885.36,5078.51],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  3.0,5.0,2.0,4.0,4.0,4.0,2.0,4.0,3.0,2.0],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  3.0,5.0,2.0,4.0,4.0,4.0,2.0,4.0,3.0,2.0],[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15  …  0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15],[4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0  …  4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[1,1,1,1,1,1,1,1,1,1  …  1,1,1,1,1,1,1,1,1,1],360600.0,[0.0 100.0 … 300.0 100.0; 100.0 0.0 … 0.0 0.0; … ; 300.0 0.0 … 0.0 700.0; 100.0 0.0 … 700.0 0.0],An

In [5]:
ta_data.travel_demand

24×24 Array{Float64,2}:
    0.0  100.0  100.0   500.0   200.0  …   100.0   400.0   300.0   100.0
  100.0    0.0  100.0   200.0   100.0        0.0   100.0     0.0     0.0
  100.0  100.0    0.0   200.0   100.0        0.0   100.0   100.0     0.0
  500.0  200.0  200.0     0.0   500.0      200.0   400.0   500.0   200.0
  200.0  100.0  100.0   500.0     0.0      100.0   200.0   100.0     0.0
  300.0  400.0  300.0   400.0   200.0  …   100.0   200.0   100.0   100.0
  500.0  200.0  100.0   400.0   200.0      200.0   500.0   200.0   100.0
  800.0  400.0  200.0   700.0   500.0      400.0   500.0   300.0   200.0
  500.0  200.0  100.0   700.0   800.0      300.0   700.0   500.0   200.0
 1300.0  600.0  300.0  1200.0  1000.0     1200.0  2600.0  1800.0   800.0
  500.0  200.0  300.0  1500.0   500.0  …   400.0  1100.0  1300.0   600.0
  200.0  100.0  200.0   600.0   200.0      300.0   700.0   700.0   500.0
  500.0  300.0  100.0   600.0   200.0      600.0  1300.0   800.0   800.0
  300.0  100.0  100.0   500

In [6]:
# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

0.0

In [7]:
# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links)

24×24 sparse matrix with 76 Int64 nonzero entries:
	[2 ,  1]  =  3
	[3 ,  1]  =  5
	[1 ,  2]  =  1
	[6 ,  2]  =  14
	[1 ,  3]  =  2
	[4 ,  3]  =  8
	[12,  3]  =  35
	[3 ,  4]  =  6
	[5 ,  4]  =  11
	[11,  4]  =  31
	⋮
	[24, 21]  =  75
	[15, 22]  =  46
	[20, 22]  =  63
	[21, 22]  =  65
	[23, 22]  =  72
	[14, 23]  =  42
	[22, 23]  =  70
	[24, 23]  =  76
	[13, 24]  =  39
	[21, 24]  =  66
	[23, 24]  =  73

In [8]:
graph.inclist, graph.is_directed, graph.nedges, graph.vertices

(Array{Graphs.Edge{Int64},1}[Graphs.Edge{Int64}[edge [1]: 1 -- 2,edge [2]: 1 -- 3],Graphs.Edge{Int64}[edge [3]: 2 -- 1,edge [4]: 2 -- 6],Graphs.Edge{Int64}[edge [5]: 3 -- 1,edge [6]: 3 -- 4,edge [7]: 3 -- 12],Graphs.Edge{Int64}[edge [8]: 4 -- 3,edge [9]: 4 -- 5,edge [10]: 4 -- 11],Graphs.Edge{Int64}[edge [11]: 5 -- 4,edge [12]: 5 -- 6,edge [13]: 5 -- 9],Graphs.Edge{Int64}[edge [14]: 6 -- 2,edge [15]: 6 -- 5,edge [16]: 6 -- 8],Graphs.Edge{Int64}[edge [17]: 7 -- 8,edge [18]: 7 -- 18],Graphs.Edge{Int64}[edge [19]: 8 -- 6,edge [20]: 8 -- 7,edge [21]: 8 -- 9,edge [22]: 8 -- 16],Graphs.Edge{Int64}[edge [23]: 9 -- 5,edge [24]: 9 -- 8,edge [25]: 9 -- 10],Graphs.Edge{Int64}[edge [26]: 10 -- 9,edge [27]: 10 -- 11,edge [28]: 10 -- 15,edge [29]: 10 -- 16,edge [30]: 10 -- 17]  …  Graphs.Edge{Int64}[edge [43]: 15 -- 10,edge [44]: 15 -- 14,edge [45]: 15 -- 19,edge [46]: 15 -- 22],Graphs.Edge{Int64}[edge [47]: 16 -- 8,edge [48]: 16 -- 10,edge [49]: 16 -- 17,edge [50]: 16 -- 18],Graphs.Edge{Int64}[edge

In [9]:
polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)])  

fcoeffs = [1, 0, 0, 0, .15]

function BPR(x)
    bpr = similar(x)
    for i = 1:length(bpr)
        bpr[i] = free_flow_time[i] * polyEval(fcoeffs, x[i]/capacity[i]) 
    end
    return bpr
end

BPR (generic function with 1 method)

In [10]:
function all_or_nothing(travel_time)
    state = []
    path = []
    x = zeros(size(start_node))

    for r=1:size(travel_demand)[1]
        # for each origin node r, find shortest paths to all destination nodes
        state = dijkstra_shortest_paths(graph, travel_time, r)

        for s=1:size(travel_demand)[2]
            # for each destination node s, find the shortest-path vector
            # load travel demand
            x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
        end
    end

    return x
end

all_or_nothing (generic function with 1 method)

In [11]:
# Finding a starting feasible solution
travel_time = BPR(zeros(number_of_links))
xl = all_or_nothing(travel_time)

76-element Array{Float64,1}:
  3800.0
  6000.0
  3800.0
  6600.0
  6000.0
  8400.0
  8600.0
  9200.0
 13700.0
  5600.0
 14400.0
 10200.0
  7000.0
     ⋮  
 13000.0
 12800.0
 25200.0
  5300.0
 13000.0
  7700.0
  5500.0
 10800.0
  5700.0
 10800.0
 12700.0
  5700.0

In [12]:
max_iter_no = 1e4
l = 1
#average_excess_cost = 1
tol = 1e-6

while l < max_iter_no
    l += 1
    
    xl_old = xl
    
    # Finding yl
    travel_time = BPR(xl)
    
    yl = all_or_nothing(travel_time)
    
    assert(yl != xl)
    
    xl = xl + (yl - xl)/l
    
    xl_new = xl
    
    relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

    if l % 2000 == 0
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
    end

    if relative_gap < tol 
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
        break
    end
    
end

l = 2000------------------------------------------------
relative_gap is 0.00010461366363755072
l = 4000------------------------------------------------
relative_gap is 3.6587185268183635e-5
l = 6000------------------------------------------------
relative_gap is 2.6262337086099878e-5
l = 8000------------------------------------------------
relative_gap is 1.6238359255744656e-5
l = 10000------------------------------------------------
relative_gap is 1.8569850206580012e-5


In [13]:
xl

76-element Array{Float64,1}:
  4496.67
  8121.5 
  4521.24
  5967.77
  8096.93
 14012.3 
 10030.4 
 14036.6 
 18010.9 
  5202.29
 18035.5 
  8797.69
 15783.6 
     ⋮   
  8620.63
 10309.6 
 18386.7 
  7001.58
  8608.18
  9661.98
  8394.92
  9626.45
  7904.09
 11112.6 
 10259.9 
  7862.79

In [14]:
outfile = open("flows_converge_Sioux.txt", "w")

write(outfile, join(("From", "to", "Volume_Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], xl[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)

In [15]:
function tapObj(linkFlowVec)
    objVal = zeros(number_of_links)
    for a = 1:number_of_links
        for i = 1:length(fcoeffs)
            objVal[a] += free_flow_time[a] * fcoeffs[i] * linkFlowVec[a]^i / capacity[a]^(i-1)
        end
    end
    return sum(objVal)
end

tapObj (generic function with 1 method)

In [16]:
tapObj(xl)

7.481792557024072e6